# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Linz  ->  H. Linz  |  ['H. Linz']
Zhang  ->  X. Zhang  |  ['Zhang']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']


Arxiv has 55 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.03796


extracting tarball to tmp_2310.03796...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.03803


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.03796/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2310.04108...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.04204


extracting tarball to tmp_2310.04204...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.04204/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgements' from 'tmp_2310.04204/acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgements' from 'tmp_2310.04204/proc-mmUniverse2023-template-NIKA2/acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.03803-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.03803) | **Investigating Protostellar Accretion-Driven Outflows Across the Mass  Spectrum: JWST NIRSpec IFU 3-5~$μ$m Spectral Mapping of Five Young  Protostars**  |
|| S. Federman, et al. -- incl., <mark>H. Beuther</mark>, <mark>H. Linz</mark> |
|*Appeared on*| *2023-10-09*|
|*Comments*| *30 pages, 11 figures*|
|**Abstract**| Investigating Protostellar Accretion (IPA) is a Cycle 1 JWST program using the NIRSpec+MIRI IFUs to obtain 2.9--28 $\mu$m spectral cubes of five young protostars with luminosities of 0.2 to 10,000 L$_{\odot}$ in their primary accretion phase. This paper introduces the NIRSpec 2.9--5.3 $\mu$m data of the inner 840-9000 au with spatial resolutions from 28-300 au. The spectra show rising continuum emission, deep ice absorption, emission from H$_{2}$, H~I, and [Fe~II], and the CO fundamental series in emission and absorption. Maps of the continuum emission show scattered light cavities for all five protostars. In the cavities, collimated jets are detected in [Fe~II] for the four $< 320$~L$_{\odot}$ protostars, two of which are additionally traced in Br-$\alpha$. Knots of [Fe~II] emission are detected toward the most luminous protostar, and knots of [FeII] emission with dynamical times of $< 30$~yrs are found in the jets of the others. While only one jet is traced in H$_2$, knots of H$_2$ and CO are detected in the jets of four protostars. H$_2$ is seen extending through the cavities showing they are filled by warm molecular gas. Bright H$_2$ emission is seen along the walls of a single cavity, while in three cavities, narrow shells of H$_2$ emission are found, one of which has an [Fe~II] knot at its apex. These data show cavities containing collimated jets traced in atomic/ionic gas surrounded by warm molecular gas in a wide-angle wind and/or gas accelerated by bow shocks in the jets. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.03796-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.03796) | **Searching for [CII] Emission from the First Sample of $z\sim 6$ OI  Absorption-Associated Galaxies with ALMA**  |
|| Y. Wu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-10-09*|
|*Comments*| *Accepted for publication in ApJS. Main text 10 pages, 5 figures*|
|**Abstract**| We report the first statistical analyses of [CII] and dust continuum observations in six strong OI absorber fields at the end of the reionization epoch obtained by the Atacama Large Millimeter/Submillimeter Array (ALMA). Combined with one [CII] emitter reported in Wu et al. (2021), we detect one OI-associated [CII] emitter in six fields. At redshifts of OI-absorbers in non-detection fields, no emitters are brighter than our detection limit within impact parameters of 50 kpc and velocity offsets between $\pm200\ {\rm km\ s^{-1}}$. The averaged [CII]-detection upper limit is $< 0.06$ Jy ${\rm km\ s^{-1}}$ (3$\sigma$), corresponding to the [CII] luminosity of $L_{\rm [CII]} <5.8\times 10^7\ L_{\odot}$ and the [CII]-based star formation rate of ${\rm SFR_{\rm [CII]}} < 5.5$ $M_\odot$ yr$^{-1}$. Cosmological simulations suggest that only $\sim10^{-2.5}$ [CII] emitters around [OI] absorbers have comparable SFR to our detection limit. Although the detection in one out of six fields is reported, an order of magnitude number excess of emitters obtained from our ALMA observations supports that the contribution of massive galaxies that caused the metal enrichment cannot be ignored. Further, we also found 14 tentative galaxy candidates with S/N of $\approx4.3$ at large impact parameters ($>50$ kpc) and having larger outflow velocities within $\pm 600$ km s$^{-1}$. If these detections are confirmed in the future, then the mechanism of pushing metals at larger distances with higher velocities needs to be further explored from the theoretical side. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.04108-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.04108) | **Orbital parameters for an ELM white dwarf with a white dwarf companion:  LAMOST J033847.06+413424.2**  |
|| H. Yuan, et al. -- incl., <mark>Zhang</mark> |
|*Appeared on*| *2023-10-09*|
|*Comments*| *12 pages, 11 figures*|
|**Abstract**| Double white dwarf systems are of great astrophysical importance in the field of gravitational wave and Type Ia supernova. While the binary fraction of CO core white dwarf is about a few percents, the extremely low mass white dwarfs are all thought to be within binary systems. In this work, we report the orbital solution of a double degenerate system: J033847.06+413424.24, an extremely low mass He core white dwarf orbiting a CO core white dwarf. With LAMOST and P200, time domain spectroscopic observations have been made and spectral atmosphere parameters are estimated to be $T_{\rm eff}\sim22500$ K and log $g\sim5.6$ dex. Combining Gaia parallax, 3D extinction, and evolution tracks, we estimate a radius of $\sim0.12$ $R_{\odot}$ and a mass of $\sim0.22$ $M_{\odot}$. With the 37 single exposure spectra, the radial velocities are measured and the orbital parameters are estimated to be $P=0.1253132(1)$ days, $K1=289\pm4$ km/s and $V_{sys}=-41\pm3$ km/s. The radial velocity based system ephemeris is also provided. The light curves from several photometric surveys show no orbital modulation. The orbital solution suggests that the invisible companion has a minimum mass of about 0.60 $M_{\odot}$ and is $\sim0.79$ $M_{\odot}$ for an inclination of $60.0^{\circ}$, indicating most probably a CO core white dwarf. The system is expected to merge in about 1 Gyr. With present period and distance ($\sim596$ pc) it can not irradiate strong enough gravitational wave for LISA. More double degenerate systems are expected to be discovered and parameterized as the LAMOST survey goes on. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.04204-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.04204) | **Exploring the interstellar medium of NGC 891 at millimeter wavelengths  using the NIKA2 camera**  |
|| S. Katsioli, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-10-09*|
|*Comments*| *To appear in Proc. of the mm Universe 2023 conference, Grenoble (France), June 2023, published by F. Mayet et al. (Eds), EPJ Web of conferences, EDP Sciences*|
|**Abstract**| In the framework of the IMEGIN Large Program, we used the NIKA2 camera on the IRAM 30-m telescope to observe the edge-on galaxy NGC 891 at 1.15 mm and 2 mm and at a FWHM of 11.1" and 17.6", respectively. Multiwavelength data enriched with the new NIKA2 observations fitted by the HerBIE SED code (coupled with the THEMIS dust model) were used to constrain the physical properties of the ISM. Emission originating from the diffuse dust disk is detected at all wavelengths from mid-IR to mm, while mid-IR observations reveal warm dust emission from compact HII regions. Indications of mm excess emission have also been found in the outer parts of the galactic disk. Furthermore, our SED fitting analysis constrained the mass fraction of the small (< 15 Angstrom) dust grains. We found that small grains constitute 9.5% of the total dust mass in the galactic plane, but this fraction increases up to ~ 20% at large distances (|z| > 3 kpc) from the galactic plane. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

288  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
